In [1]:
import json
with open('dataset.json', 'r') as f:
    dataset = json.load(f)
dataset.keys()

dict_keys(['columns', 'index', 'data'])

In [2]:
with open('data.json', 'r') as f:
    labels = json.load(f)
labels

{'Boy': 0,
 'Can': 1,
 'Eat': 2,
 'Fine': 3,
 'Girl': 4,
 'Help': 5,
 'How': 6,
 'Hungry': 7,
 'I': 8,
 'Mother': 9,
 'Namaste': 10,
 'Name': 11,
 'Parents': 12,
 'Sister': 13,
 'Sleep': 14,
 'This': 15,
 'You': 16}

In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM, Conv1D, MaxPooling1D

In [19]:
dataset=dataset['data']
dataset[:5]

[['Boy',
  [0.6076942395,
   0.0852121783,
   0.2623527293,
   0.5865284134,
   0.0792704998,
   0.267338082,
   0.5749486898,
   0.0585460731,
   0.2692916729,
   0.5692403065,
   0.0361146114,
   0.2699350342,
   0.5655733293,
   0.0194674883,
   0.2699414286,
   0.5988154448,
   0.0390192262,
   0.2700981361,
   0.5877198291,
   0.0105843135,
   0.2720279156,
   0.5746693697,
   0.0027865703,
   0.272286512,
   0.5642671079,
   0.0026097212,
   0.2719373752,
   0.6089250739,
   0.0343905998,
   0.2677049395,
   0.5937953162,
   0.0074807682,
   0.2693170124,
   0.5789668017,
   0.0002214472,
   0.2692496329,
   0.5678925595,
   0.0,
   0.2690374232,
   0.6134769107,
   0.0329757279,
   0.2654027912,
   0.5984557606,
   0.0080899854,
   0.2669032778,
   0.5844174215,
   0.0021906494,
   0.2666853588,
   0.5736784062,
   0.0021796483,
   0.2663043722,
   0.6136828589,
   0.0331092057,
   0.2633471756,
   0.6012492059,
   0.0132860406,
   0.2648700107,
   0.5895634701,
   0.0065332069,

In [20]:
len(dataset[0][3])

198

In [6]:
len(dataset)

722

In [7]:
len(dataset[0])

20

In [34]:
Y={'Class Label': []}
data=[]

In [35]:
for i in range(len(dataset)):
    Y['Class Label']+=[labels[dataset[i][0]]]
    l=np.array(dataset[1][1:])
    l=l.flatten()
    l=l.tolist()
    print
    data.append(l)
        

In [50]:
data=np.array(data)
data[0]

array([1.        , 0.87880973, 0.43762634, ..., 0.41683706, 0.46658458,
       0.32171747])

In [39]:
# data.shape
type(data)

numpy.ndarray

In [40]:
Y

{'Class Label': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,


In [41]:
Y=pd.DataFrame(Y)
Y

,Class Label
0,0
1,0
2,0
3,0
4,0
...,...
717,16
718,16
719,16
720,16


In [14]:
Y=to_categorical(Y).astype(int)
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [42]:
Y.shape

(722, 1)

In [43]:
x_train, x_test, y_train, y_test = train_test_split(data, Y, test_size=0.2, random_state=42)

In [44]:
x_train.shape

(577, 3762)

In [45]:
y_train.shape

(577, 1)